In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import hydrant.ObjectiveFunctions.ObjectiveFunctions as Obj

In [2]:
# Example usage
# Create example data for observation
time_obs = pd.date_range(start='1950-01-01', end='2020-12-31', freq='D')
ids_obs = np.arange(1, 101)
data_obs = np.random.rand(len(time_obs), len(ids_obs))
# add np.nan values to emulate missing values in observation
data_obs[:,1] = np.nan
data_obs[:,-1] = np.nan
data_obs[1,:] = np.nan
data_obs[-1,:] = np.nan
# convert to dataframe
observation = xr.Dataset({'Discharge_obs': (['time', 'COMID'], data_obs)},
                         coords={'time': time_obs, 'COMID': ids_obs})
observation

<xarray.Dataset>
Dimensions:        (time: 25933, COMID: 100)
Coordinates:
  * time           (time) datetime64[ns] 1950-01-01 1950-01-02 ... 2020-12-31
  * COMID          (COMID) int64 1 2 3 4 5 6 7 8 9 ... 93 94 95 96 97 98 99 100
Data variables:
    Discharge_obs  (time, COMID) float64 0.4527 nan 0.2151 ... nan nan nan

In [3]:
# Create example data for simulation
time_sim = pd.date_range(start='1950-01-01 12:00:00', end='2100-12-31 12:00:00', freq='D')
ids_sim = np.arange(50, 151)
data_sim = np.random.rand(len(time_sim), len(ids_sim))
simulation = xr.Dataset({'Discharge_sim': (['time', 'SegId'], data_sim)},
                        coords={'time': time_sim, 'SegId': ids_sim})
simulation

<xarray.Dataset>
Dimensions:        (time: 55152, SegId: 101)
Coordinates:
  * time           (time) datetime64[ns] 1950-01-01T12:00:00 ... 2100-12-31T1...
  * SegId          (SegId) int64 50 51 52 53 54 55 ... 145 146 147 148 149 150
Data variables:
    Discharge_sim  (time, SegId) float64 0.2735 0.4643 0.2291 ... 0.05059 0.2197

In [4]:
# Call the ObjectiveFunction with the example datasets
Objective = Obj.ObjectiveFunction(observation,
                                  simulation,
                                  info_obs={'var': 'Discharge_obs', 'var_id': 'COMID', 'dim_id': 'COMID', 'var_time': 'time', 'dim_time': 'time'},
                                  info_sim={'var': 'Discharge_sim', 'var_id': 'SegId', 'dim_id': 'SegId', 'var_time': 'time', 'dim_time': 'time'},
                                  TimeStep = 'daily')

# Print the result
Objective

<xarray.Dataset>
Dimensions:  (time: 25933, ID: 51)
Coordinates:
  * time     (time) datetime64[ns] 1950-01-01 1950-01-02 ... 2020-12-31
  * ID       (ID) int64 50 51 52 53 54 55 56 57 58 ... 93 94 95 96 97 98 99 100
Data variables:
    obs      (time, ID) float64 0.4239 0.2479 0.7285 0.1642 ... nan nan nan nan
    sim      (time, ID) float64 0.2735 0.4643 0.2291 0.1239 ... 0.7 0.618 0.231
    KGE      (ID) float64 0.005217 -0.0003327 0.00501 ... 0.002945 0.009127 nan
    NSE      (ID) float64 -0.994 -1.001 -0.9956 -0.9936 ... -0.996 -0.9884 nan
    RMSE     (ID) float64 0.407 0.4078 0.4059 0.4098 ... 0.4065 0.4066 nan